<a href="https://colab.research.google.com/github/Python-Financial-Analyst/pyfian_dev/blob/main/notebooks/fixed_income/02_bullet_bonds.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

# Bullet Bonds

## Learning Objectives

- Understand what is a Bullet Bond
- Understand different day count conventions
- Understand accrued interest
- Learn about discount rate



In [1]:
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet
import pandas as pd
from IPython.display import display

# Example bond
bond = FixedRateBullet(
    "2020-01-01", "2030-01-01", 5, 1, notional=100, day_count_convention="30/365"
)


def display_bond_cash_flows(cash_flow_dict):
    df = pd.DataFrame(
        sorted(cash_flow_dict.items(), key=lambda x: x[0]),
        columns=["Date", "Cash Flow Value"],
    )
    df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")
    df["Cash Flow Value"] = df["Cash Flow Value"].round(2)

    # Display neatly formatted DataFrame
    display(
        df.style.set_table_styles(
            [
                {
                    "selector": "th",
                    "props": [
                        ("color", "white"),
                        ("font-weight", "bold"),
                        ("text-align", "center"),
                    ],
                },
                {"selector": "td", "props": [("text-align", "center")]},
            ]
        ).set_caption("Bond Cash Flows Table")
    )


# Example usage
display_bond_cash_flows(bond.payment_flow)

,Date,Cash Flow Value
0,2021-01-01,5.000000
1,2022-01-01,5.000000
2,2023-01-01,5.000000
3,2024-01-01,5.000000
4,2025-01-01,5.000000
5,2026-01-01,5.000000
6,2027-01-01,5.000000
7,2028-01-01,5.000000
8,2029-01-01,5.000000
9,2030-01-01,105.000000


## Bond Day-Counting Conventions

These rules decide how to count days for calculating bond interest:

- **30/360**: Counts each month as 30 days, year as 360 days. Simple but less accurate. Used in some corporate bonds.
- **30E/360**: Like 30/360, but if a date is the 31st, it’s counted as the 30th. Used in Eurobonds.
- **Actual/Actual**: Counts real days in the period and year (365 or 366). Most accurate. Used in US Treasuries (ISDA) or international bonds (ICMA).
- **Actual/360**: Counts real days in the period, but uses a 360-day year. Common in money markets like loans.
- **Actual/365**: Counts real days in the period, uses a 365-day year. Used in some bond markets.
- **30/365**: Counts each month as 30 days, year as 365 days. Rare, used in specific financial contracts.
- **Actual/Actual-Bond**: Same as Actual/Actual (ICMA), counts real days in period and coupon period (e.g., half-year). Used in bond markets.

In [2]:
from pyfian.fixed_income.money_market_instruments import (
    TreasuryBill,
    CertificateOfDeposit,
    CommercialPaper,
    BankersAcceptance,
)

# ===== Adjustable Parameters =====
issue_dt = "2025-01-31"
maturity = "2030-01-31"
settlement_date = "2027-01-01"
notional = 100
price = 85
cpn = 0  # only used for CD
conventions = ["30/360", "30e/360", "actual/360", "actual/365", "30/365"]


instrument_classes = [
    TreasuryBill,
    CertificateOfDeposit,
    CommercialPaper,
    BankersAcceptance,
]

# ===== Build Data =====
data = []

for cls in instrument_classes:
    for conv in conventions:
        try:
            # Create instrument with specific day count convention
            kwargs = {
                "issue_dt": issue_dt,
                "maturity": maturity,
                "settlement_date": settlement_date,
                "notional": notional,
                "price": price,
                "day_count_convention": conv,
            }

            # Only CD uses coupon
            if cls == CertificateOfDeposit:
                kwargs["cpn"] = cpn

            inst = cls(**kwargs)

            data.append(
                {
                    "Instrument": cls.__name__,
                    "Day Count Convention": conv,
                    "Yield Convention": inst.yield_calculation_convention,
                    "Price": inst.get_price(),
                    "Notional": inst.notional,
                    "YTM": inst.get_yield_to_maturity(),
                }
            )
        except Exception:
            # Handle invalid combinations
            data.append(
                {
                    "Instrument": cls.__name__,
                    "Day Count Convention": conv,
                    "Yield Convention": getattr(
                        cls, "yield_calculation_convention", ""
                    ),
                    "Price": None,
                    "Notional": notional,
                    "YTM": None,
                    # "Error": str(e)
                }
            )

# Build DataFrame
df = pd.DataFrame(data)

# Display
display(df)

,Instrument,Day Count Convention,Yield Convention,Price,Notional,YTM
0,TreasuryBill,30/360,Discount,85,100,0.048649
1,TreasuryBill,30e/360,Discount,85,100,0.048693
2,TreasuryBill,actual/360,Discount,85,100,0.047957
3,TreasuryBill,actual/365,Discount,85,100,0.048623
4,TreasuryBill,30/365,Discount,85,100,0.049369
5,CertificateOfDeposit,30/360,Add-On,85,100,0.057234
6,CertificateOfDeposit,30e/360,Add-On,85,100,0.057285
7,CertificateOfDeposit,actual/360,Add-On,85,100,0.056420
8,CertificateOfDeposit,actual/365,Add-On,85,100,0.057204
9,CertificateOfDeposit,30/365,Add-On,85,100,0.058081


In [ ]:
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet


def compare_accrued_interest(
    issue_date, maturity, coupon_rate, freq, notional, conventions, valuation_date
):
    """
    Compare accrued interest for different day count conventions
    at a given valuation date.
    """
    results = []

    for conv in conventions:
        bond = FixedRateBullet(
            issue_date,
            maturity,
            coupon_rate,
            freq,
            notional=notional,
            day_count_convention=conv,
        )
        # Compute accrued interest for given valuation date
        accrued = bond.accrued_interest(valuation_date)

        results.append(
            {
                "Convention": conv,
                "Valuation Date": pd.to_datetime(valuation_date).strftime("%Y-%m-%d"),
                "Accrued Interest": round(accrued, 4),
            }
        )

    # Create and display pandas table
    df = pd.DataFrame(results)
    df = df[["Convention", "Valuation Date", "Accrued Interest"]]

    display(
        df.style.set_table_styles(
            [
                {
                    "selector": "th",
                    "props": [
                        ("background-color", "#0072B2"),
                        ("color", "white"),
                        ("font-weight", "bold"),
                        ("text-align", "center"),
                    ],
                },
                {"selector": "td", "props": [("text-align", "center")]},
            ]
        ).set_caption("Comparison of Accrued Interest by Day Count Convention")
    )


# ✅ Example usage
compare_accrued_interest(
    issue_date="2020-01-01",
    maturity="2025-01-01",
    coupon_rate=5,
    freq=2,
    notional=100,
    conventions=[
        "30/360",
        "30e/360",
        "actual/360",
        "actual/365",
        "30/365",
        "actual/actual-Bond",
    ],
    valuation_date="2022-03-15",
)

,Convention,Valuation Date,Accrued Interest
0,30/360,2022-03-15,1.027800
1,30e/360,2022-03-15,1.027800
2,actual/360,2022-03-15,1.013900
3,actual/365,2022-03-15,1.000000
4,30/365,2022-03-15,1.013700
5,actual/actual-Bond,2022-03-15,1.008300


In [ ]:
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet


def compare_ytm_day_count_conventions(
    issue_date, maturity, coupon_rate, freq, notional, conventions, market_price
):
    results = []

    for conv in conventions:
        # Create bond with given day count convention
        bond = FixedRateBullet(
            issue_date,
            maturity,
            coupon_rate,
            freq,
            notional=notional,
            day_count_convention=conv,
        )

        # Compute yield to maturity given market price
        ytm = bond.yield_to_maturity(market_price)

        results.append(
            {
                "Convention": conv,
                "Coupon (%)": coupon_rate,
                "Frequency": freq,
                "Market Price": round(market_price, 4),
                "YTM (%)": round(ytm, 6),
            }
        )

    # Create styled pandas table
    df = pd.DataFrame(results)

    display(
        df.style.set_table_styles(
            [
                {
                    "selector": "th",
                    "props": [
                        ("background-color", "#0072B2"),
                        ("color", "white"),
                        ("font-weight", "bold"),
                        ("text-align", "center"),
                    ],
                },
                {"selector": "td", "props": [("text-align", "center")]},
            ]
        ).set_caption("Comparison of YTM by Day Count Convention")
    )


compare_ytm_day_count_conventions(
    issue_date="2020-01-01",
    maturity="2025-01-01",
    coupon_rate=5,
    freq=2,
    notional=100,
    conventions=["30/360", "30e/360", "actual/360", "actual/365", "actual/actual-Bond"],
    market_price=98.5,
)

,Convention,Coupon (%),Frequency,Market Price,YTM (%)
0,30/360,5,2,98.500000,0.053458
1,30e/360,5,2,98.500000,0.053458
2,actual/360,5,2,98.500000,0.053458
3,actual/365,5,2,98.500000,0.053458
4,actual/actual-Bond,5,2,98.500000,0.053458


## Calculating Bond Price from Discount Curve

In order to price a bond correctly, we just need to compute the present value of the future cash flows. The cash flows consist of the periodic coupon payments and the face value paid at maturity, in the case of the bullet bond. If the bond amortizes before maturity, these amortization payments must also be included in the bond cash flows.

## Discount Rate

In order to discount the cash flows we need to find the discount rate, which is found using the exponential function: $P = F \times e^{-rt}$.

In [ ]:
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet

# -------------------------------
# Bond example
# -------------------------------
bond_discount_example = FixedRateBullet(
    issue_dt="2026-01-01",
    maturity="2031-01-01",
    cpn=5,
    cpn_freq=1,
    notional=100,
    yield_to_maturity=0.05,
    settlement_date="2027-01-01",
)

# Compute price (no need for yield_rate argument)
price = bond_discount_example.get_price()
print(f"Bond Price = {price:.2f}")

Bond Price = 99.78


## Duration

Duration measures a bond's price sensitivity to changes in interest rates and represents the weighted average time to receive the bond's cash flows. Mathematically, duration approximates the percentage change in a bond's price for a small change in yield, expressed as:

$$D = -\frac{1}{P} \cdot \frac{dP}{dr}$$

The negative sign reflects the inverse relationship between bond prices and yields.

In [ ]:
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet

bond_duration_example = FixedRateBullet(
    issue_dt="2020-01-01",
    maturity="2025-01-01",
    cpn=5,
    cpn_freq=1,
    notional=1000,
    yield_to_maturity=5,
    settlement_date="2021-01-01",
)

### Macaulay Duration

Macaulay duration is the weighted average time until a bond's cash flows are received, expressed in years. It is calculated as:

$$\frac{\sum_{t=1}^n \frac{t \cdot C_t}{(1 + y)^t} + \frac{n \cdot FV}{(1 + y)^n}}{P}$$

where:
- $C_t$: Cash flow (coupon payment) at time $t$
- $FV$: Face value of the bond
- $y$: Yield to maturity (per period)
- $n$: Number of periods until maturity
- $P$: Present value (current price) of the bond

For a zero-coupon bond, since there are no coupon payments ($C_t = 0$), the Macaulay duration simplifies to the time to maturity, $T - t$, where $T$ is the maturity date and $t$ is the current date.

In [ ]:
macaulay_duration = bond_duration_example.macaulay_duration()
print(macaulay_duration)

1.1175133907


### Modified Duration

Modified duration directly measures the percentage change in a bond's price for a 1% change in yield. It is derived from the Macaulay duration as:

$$D^{\text{Mod}} = \frac{D^{\text{MC}}}{1 + \frac{y}{k}}$$

where:
- $y$: Annual yield to maturity
- $k$: Number of compounding periods per year (e.g., $k = 2$ for semi-annual, $k = 1$ for annual)

For a zero-coupon bond with annual compounding ($k = 1$), the modified duration is:

$$D^{\text{Mod}} = \frac{T - t}{1 + y}$$


In [ ]:
modified_duration = bond_duration_example.modified_duration()

print(modified_duration)

0.3192895402



## Convexity

Convexity measures the sensitivity of a bond’s duration to changes in interest rates and captures the non-linear relationship between bond prices and yields. It is the second derivative of the bond’s price with respect to the yield, scaled by the price:

$$C = \frac{1}{P} \cdot \frac{d^2P}{dr^2}$$

For a zero-coupon bond with annual compounding, the convexity is:

$$C = \frac{(T - t)(T - t + 1)}{(1 + y)^2}$$

In [ ]:
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet

convexity_example_bond = FixedRateBullet(
    issue_dt="2020-01-01",
    maturity="2025-01-01",
    cpn=5,
    cpn_freq=1,
    notional=1000,
    yield_to_maturity=5,
    settlement_date="2021-01-01",
)

convexity = convexity_example_bond.convexity()

print(convexity)

0.162101523
